# Hierarchical Time Series Forecasting Pipeline

This notebook demonstrates our pipeline for comparison of model and reconciliation approaches in time-series forecasting as part of our research/consulting project with BASF. An interactive analysis of the results is provided separately in the notebook title results_analysis.ipynb and deeper discussion is available in our slides or paper. When reading, it may help to keep in mind that throughout this project, EBIT is the target variable and by default lies in the middle of the data's hierarchy.

This pipeline includes:
- Multiple forecasting models
- Multiple hierarchical reconciliation methods
- Option for feature engineering and added covariates
- Option to switch hierarchical paths as outlined in paper/slides
- Standardized model comparison framework

## Setup and Configuration

### Package Setup

In [44]:
#may need to install darts depending on environment
#!python -m pip install darts

In [45]:
import pandas as pd
from darts.models import (Prophet, LinearRegressionModel, ARIMA,  ExponentialSmoothing, XGBModel,  NBEATSModel, GlobalNaiveAggregate, NaiveDrift)
from darts.dataprocessing.transformers import MinTReconciliator, BottomUpReconciliator, TopDownReconciliator
from utils import (get_winners,get_best_per_series, load_data, apply_hierarchy, compare_models_multivariate, compare_models_reconciliated, compare_models_univariate)
import matplotlib.pyplot as plt
import warnings
import logging

Ignore excessive noise by silencing some warnings.

In [46]:
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)

warnings.filterwarnings("ignore")
logging.disable(logging.CRITICAL)



### Hierarchy Configuration

Here you can change which subset of the data and corresponding hierarchical structure to use. 

The options are:
- 'var0': the whole hierarchy, includes both other paths
- 'var1': EBIT - DA = EBITDA
- 'var2': Net Sales  - Variable Costs = CM and CM - Fix Costs = EBIT

In [47]:

hvar = 'var2'

### Data Import and Prep

Import data, apply hierarchy, and split into test-validation sets. Versions of the data both with and without the hierarchy are required.

In [48]:
df = load_data(file_path='data/SampleHierForecastingBASF_share.xlsx')
series, target, covariates, hierarchy = apply_hierarchy(df, hvar=hvar)
train, val = target[:-24], target[-24:-12]
hierarchical_train = series[:-24]
hierarchical_val = series[-24:-12]

## Model Comparison

Let's start by defining the models we will look at. We have univariate and multivariate baselines represented with uni_ and multi_models to test. Feel free to add additional models to test. Models are expected to be from the darts package. The NBEATS parameters here were derived using Optuna and can also be adjusted as needed. For implementation details of the hyperparameter tuning process check hpo.py.

In [49]:
uni_models_to_test = [
    ARIMA(q=1),
    ExponentialSmoothing(),
    Prophet(),
    NBEATSModel(input_chunk_length=36,output_chunk_length=24, dropout= 0.11891699976631348, n_epochs=27, batch_size=128),
    LinearRegressionModel(lags=12),
    XGBModel(lags=12)
    ]
multi_models_to_test  = [
    NBEATSModel(input_chunk_length=36,output_chunk_length=24, dropout= 0.11891699976631348, n_epochs=27, batch_size=128),
    LinearRegressionModel(lags=12),
    XGBModel(lags=12)
    ]


Additionally, we implement here the multi-model approach outlined in our paper. From a set of suitably simple models, the best performing one is selected on a series-by-series basis. The combination of these predictions is considered a model of its own.

In [50]:
simple_models_to_test = [
    ARIMA(q=1),
    ExponentialSmoothing(),
    Prophet(),
    #LinearRegressionModel(lags=12),
    NaiveDrift(),
    GlobalNaiveAggregate(input_chunk_length=3, output_chunk_length=3),
    GlobalNaiveAggregate(input_chunk_length=1, output_chunk_length=1),
    GlobalNaiveAggregate(input_chunk_length=12, output_chunk_length=12),
    ]

best_per_series = get_best_per_series(hierarchical_train, hierarchical_val,models=simple_models_to_test)
best_per_series=best_per_series.with_hierarchy(hierarchy)

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 266.37it/s]
ContributionMargin1: ARIMA
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 167.63it/s]
-FixCosts: ExponentialSmoothing
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 322.12it/s]
NetSales: ARIMA
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 294.79it/s]
-VariableCosts: ARIMA
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 274.60it/s]
EBIT: ARIMA


Now, we have the actual comparison step. We pass all of our models to the compare functions. These return the fitted models and predictions for all passed model names. 

In [51]:
# compare univariate baseline models, uncreconciled multivariate models, reconciled multivariate models
# includes multi-model in the multivariate models
fittedbaselinemodels, univariate_predictions = compare_models_univariate(train, val, uni_models_to_test)
unreconciled_models, unreconciled_predictions = compare_models_multivariate(hierarchical_train, hierarchical_val, multi_models_to_test)
unreconciled_predictions['Multi Model']=best_per_series

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 61.01it/s]
MAE for ARIMA, Uni: 10593.30
MAPE for ARIMA, Uni: 220.89
RMSE for ARIMA, Uni: 12526.19
R^2 for ARIMA, Uni: 0.80
SMAPE for ARIMA, Uni: 81.98

MAE for ExponentialSmoothing, Uni: 9987.80
MAPE for ExponentialSmoothing, Uni: 188.91
RMSE for ExponentialSmoothing, Uni: 11312.62
R^2 for ExponentialSmoothing, Uni: 0.83
SMAPE for ExponentialSmoothing, Uni: 84.67

MAE for Prophet, Uni: 12980.65
MAPE for Prophet, Uni: 103.01
RMSE for Prophet, Uni: 16242.64
R^2 for Prophet, Uni: 0.66
SMAPE for Prophet, Uni: 117.49

MAE for NBEATSModel, Uni: 7369.67
MAPE for NBEATSModel, Uni: 98.65
RMSE for NBEATSModel, Uni: 8942.28
R^2 for NBEATSModel, Uni: 0.90
SMAPE for NBEATSModel, Uni: 87.57

MAE for LinearRegression, Uni: 10010.87
MAPE for LinearRegression, Uni: 225.42
RMSE for LinearRegression, Uni: 12464.43
R^2 for LinearRegression, Uni: 0.80
SMAPE for LinearRegression, Uni: 78.46

MAE for XGBRegressor, Uni: 10149.12
MAPE for XGBRegressor

Let's define our reconciliation methods of choice. Some must be fit before they can be applied. We check performance after reconciliation using the compare_models_reconciliated function, which is similar to the functions used above.

In [52]:

reconciliator0 = MinTReconciliator(method="ols")
reconciliator0.fit(series[:-24])
reconciliator1 = TopDownReconciliator()
reconciliator1.fit(series[:-24])
reconciliator2 = BottomUpReconciliator()
reconciliators = {reconciliator0 : 'MinT',
                  reconciliator1 : 'Top Down',
                  reconciliator2 : 'Bottom Up'}


In [53]:

reconciliatedpredictions = compare_models_reconciliated(data=hierarchical_train, val=val['EBIT'], models=unreconciled_predictions, reconciliators=reconciliators)

Testing NBEATSModel
Testing LinearRegression
Testing XGBRegressor
Testing Multi Model
MAE for NBEATSModel, MinT: 16918.10
MAPE for NBEATSModel, MinT: 203.98
RMSE for NBEATSModel, MinT: 20428.10
R^2 for NBEATSModel, MinT: 0.46
SMAPE for NBEATSModel, MinT: 100.96

MAE for NBEATSModel, Top Down: 19992.12
MAPE for NBEATSModel, Top Down: 298.64
RMSE for NBEATSModel, Top Down: 23752.01
R^2 for NBEATSModel, Top Down: 0.27
SMAPE for NBEATSModel, Top Down: 111.13

MAE for NBEATSModel, Bottom Up: 20423.94
MAPE for NBEATSModel, Bottom Up: 222.80
RMSE for NBEATSModel, Bottom Up: 26406.72
R^2 for NBEATSModel, Bottom Up: 0.10
SMAPE for NBEATSModel, Bottom Up: 97.42

MAE for LinearRegression, MinT: 13287.55
MAPE for LinearRegression, MinT: 268.58
RMSE for LinearRegression, MinT: 14988.62
R^2 for LinearRegression, MinT: 0.71
SMAPE for LinearRegression, MinT: 89.97

MAE for LinearRegression, Top Down: 13287.55
MAPE for LinearRegression, Top Down: 268.58
RMSE for LinearRegression, Top Down: 14988.62
R^2

## Export Results

Evaluations of accuracy can be found in the files titled results_var. The raw predictions can be found in the files predictions_var. For further analysis of these files, check the other notebook provided.

In [54]:
merged={**unreconciled_predictions,
**reconciliatedpredictions,}

merged = {k:v['EBIT'] for k,v in merged.items()}

merged = {**univariate_predictions,**merged}

In [55]:
dfs = []
for model_name, ts in merged.items():
    df = ts.to_dataframe().reset_index()  
    df.columns = ["Date", "Predictions"]   
    df["Name"] = model_name
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

merged_df = merged_df[["Date", "Name", "Predictions"]]
merged_df['Date']=pd.to_datetime(merged_df['Date'])

In [56]:
quarterly, summary = get_winners(merged_df,val['EBIT'])
summary.to_csv('output/results_'+hvar)
merged_df.to_csv('output/predictions_'+hvar)